In [ ]:
After sucessfully loeading all tables into a database and merging them into one table, this table now will be cleaned and transformed for further analysis.

But at first the required modules and credentials are loaeded, and the MySQL connection is set up. 

In [ ]:
import pandas as pd
import json
import mysql.connector

In [ ]:
#get credentials for MySQL/dbms
with open(r'C:\Users\mathi\Desktop\Datenanalyse\Credentials\mysql.json') as json_file:
    db_credentials = json.load(json_file)['MySQL']

#credentials as variables
host='localhost'
user=db_credentials['user']
password=db_credentials['password']
database='feederwatch'

#etablish connection to the dbms
mydb = mysql.connector.connect(
  host=host,
  user=user,
  password=password)

mycursor = mydb.cursor()

#select database
query=f'USE {database}'
mycursor.execute(query)

To figure out, what to do, we'll first inspect the table structure.

In [ ]:
query='DESCRIBE observations'
mycursor.execute(query)
for line in mycursor:
    print(line)